In [2]:
#Import libraries for data handling and matrix algebra
import pandas as pd
import numpy as np 

In [3]:
#Load in datafile -- this is already pre-processed the same way we did in MATLAB
df = pd.read_pickle('dir_here')

In [5]:
'''Entropy is computed from Daeyoels paper using 3 choice sequences as a sample. 
This slide_window function basically grabs the triplet groups from the choice vector array'''
def slide_window(sequence,win_size,step=1):
    
    #Define # of triplets to grab using the window size and step (1 since we don't want to skip values)
    num_chunks = int((len(sequence) - win_size/step) + 1)
    
    #A for loop which returns the next triplet everytime the function is called
    for i in range(0,num_chunks*step,step):
        yield sequence[i:i+win_size] 

In [6]:
#Compute entropy function
def entropy_choice(choice,win_size=3):
    
    
    #This is an iterator which stores the slide_window function
    win_iterate = slide_window(choice,win_size) 

    #Set up a dictionary to store all unique triplets 
    entropy_dict = dict() 
    
    #Iterate or loop through the windows provided by slide_window function
    for chunk in win_iterate:
        '''Chunk is a triplet returned from the slide_window function, we want the total count
        of all unique triplets 2^3 possible combinations'''
        
        #Method of counting unique values
        
        #If we've see the triplet before, add +1 to it's total
        if tuple(chunk) in entropy_dict: 
            entropy_dict[tuple(chunk)] += 1
        else: 
            #If not, then create a new dictionary entry for the triplet and set it to 1
            entropy_dict[tuple(chunk)] = 1
    
    #Biased entropy is a sum of all entropies of each triplet
    #Refer to Daeyoel's paper for formula
    biased_entropy = 0
    
    #For each unique triplet
    for key in entropy_dict:
                
        #Calculate the un-corrected entropy (sum of p * log2p) for each of the unique choice triplets
        #Then accumulate the sum
        biased_entropy += entropy_dict[key]/sum(entropy_dict.values()) \
        * np.log2(entropy_dict[key]/sum(entropy_dict.values()))
    
    #Add correction factor proposed by Miller (1955), not sure why 1.3863N is a factor when in Miller 2N is 
    #the value used... talk to Daeyoel
    unbiased_entropy = -biased_entropy + ((2**win_size - 1) / (1.3863 * sum(entropy_dict.values())))
    return unbiased_entropy
    
    


In [7]:
#Here we loop through each of the row entries in the dataframe and compute entropy
dict_entropy = [] 
count = 0
#Loop through direction and color choice column rows
for row in df[['dir_choice','col_choice']].itertuples(index=False):
    #Create a dictionary to create a dataframe
    dict_entropy.append({'dir_entropy' : entropy_choice(row[0]), 'col_entropy' : entropy_choice(row[1])})
    


In [ ]:
#Add the entropy for direction and choice columns to original dataframe and display
df_ent = df.join(pd.DataFrame.from_dict(dict_entropy))


In [ ]:
#Get eye trials
df_ent = df_ent[df_ent['eob'] == 'eye']

#Just a quick look at statistics of entropy across groups
df_grpmeds = df_ent.groupby(by=['group','meds'])
df_grpmeds.describe()

In [10]:
#Import plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sn 
import importlib
%matplotlib inline

In [11]:
#Set styles used for plotting
sn.set_style('ticks')
sn.set_context('talk')

In [ ]:
#Plot using Seaborn's violin plot
plt.figure(figsize=(18,10))
sn.violinplot('group','dir_entropy',data=df_ent,hue='meds',split=True,width=1,palette='deep')
sn.despine(offset=15,trim=True)

In [ ]:
#For color choice
plt.figure(figsize=(18,10))
sn.violinplot('group','col_entropy',data=df_ent,hue='meds',split=True,width=1,palette='deep')
sn.despine(offset=15,trim=True)